<a href="https://colab.research.google.com/github/anastefan176/ULEZ/blob/main/Regression_Discontinuity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import pylab
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')

pylab.rcParams['figure.figsize'] = (10., 8.)

In [6]:
from google.colab import files
uploaded = files.upload()

data = pd.read_csv('LaqnData (2).csv')
data.head()

Saving LaqnData (2).csv to LaqnData (2).csv


,Site,Species,ReadingDateTime,Value,Units,Provisional or Ratified
0,HR1,NO2,01/01/2020 00:00,33.6,ug m-3,R
1,HR1,NO2,02/01/2020 00:00,15.9,ug m-3,R
2,HR1,NO2,03/01/2020 00:00,13.2,ug m-3,R
3,HR1,NO2,04/01/2020 00:00,19.7,ug m-3,R
4,HR1,NO2,05/01/2020 00:00,14.1,ug m-3,R


In [7]:
data = pd.read_csv('LaqnData (2).csv')
data.columns = ["site", "pollutant", "datetime", "Value", "Units", "status"]

data["datetime"] = pd.to_datetime(data["datetime"], dayfirst=True, errors="coerce")

data = data.dropna(subset=["datetime", "Value"])

data = data[data["status"] == "R"]

data["Value"] = pd.to_numeric(data["Value"], errors="coerce")

data = data[(data["Value"] >= 0) & (data["Value"] <= 200)]

print(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 6330 entries, 0 to 10961
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   site       6330 non-null   object        
 1   pollutant  6330 non-null   object        
 2   datetime   6330 non-null   datetime64[ns]
 3   Value      6330 non-null   float64       
 4   Units      6330 non-null   object        
 5   status     6330 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 346.2+ KB
None


,site,pollutant,datetime,Value,Units,status
0,HR1,NO2,2020-01-01,33.6,ug m-3,R
1,HR1,NO2,2020-01-02,15.9,ug m-3,R
2,HR1,NO2,2020-01-03,13.2,ug m-3,R
3,HR1,NO2,2020-01-04,19.7,ug m-3,R
4,HR1,NO2,2020-01-05,14.1,ug m-3,R


In [8]:
from google.colab import files
uploaded = files.upload()

data2 = pd.read_csv('LaqnData (3).csv')
data.head()

Saving LaqnData (3).csv to LaqnData (3).csv


,site,pollutant,datetime,Value,Units,status
0,HR1,NO2,2020-01-01,33.6,ug m-3,R
1,HR1,NO2,2020-01-02,15.9,ug m-3,R
2,HR1,NO2,2020-01-03,13.2,ug m-3,R
3,HR1,NO2,2020-01-04,19.7,ug m-3,R
4,HR1,NO2,2020-01-05,14.1,ug m-3,R


In [9]:
data2 = pd.read_csv('LaqnData (3).csv')


data2.columns = ["site", "pollutant", "datetime", "Value", "Units", "status"]
data2["datetime"] = pd.to_datetime(data2["datetime"], dayfirst=True, errors="coerce")
data2 = data2.dropna(subset=["datetime", "Value"])
data2 = data2[data2["status"] == "R"]
data2["Value"] = pd.to_numeric(data2["Value"], errors="coerce")
data2 = data2[(data2["Value"] >= 0) & (data2["Value"] <= 200)]

print("Second file (data2) after cleaning:")
print(data2.info())
data2.head()

Second file (data2) after cleaning:
<class 'pandas.core.frame.DataFrame'>
Index: 6646 entries, 0 to 9864
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   site       6646 non-null   object        
 1   pollutant  6646 non-null   object        
 2   datetime   6646 non-null   datetime64[ns]
 3   Value      6646 non-null   float64       
 4   Units      6646 non-null   object        
 5   status     6646 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 363.5+ KB
None


,site,pollutant,datetime,Value,Units,status
0,MY1,NO2,2020-01-01,51.3,ug m-3,R
1,MY1,NO2,2020-01-02,51.5,ug m-3,R
2,MY1,NO2,2020-01-03,43.1,ug m-3,R
3,MY1,NO2,2020-01-04,60.5,ug m-3,R
4,MY1,NO2,2020-01-05,57.1,ug m-3,R


In [10]:
combined = pd.concat([data, data2], ignore_index=True)

In [11]:
combined.shape
combined.head()

,site,pollutant,datetime,Value,Units,status
0,HR1,NO2,2020-01-01,33.6,ug m-3,R
1,HR1,NO2,2020-01-02,15.9,ug m-3,R
2,HR1,NO2,2020-01-03,13.2,ug m-3,R
3,HR1,NO2,2020-01-04,19.7,ug m-3,R
4,HR1,NO2,2020-01-05,14.1,ug m-3,R


In [12]:
combined["site"].unique()

array(['HR1', 'HV1', 'ST4', 'LB4', 'MY1', 'BG2', 'BL0', 'TH2', 'BX1'],
      dtype=object)

In [13]:
print("data sites:", data["site"].unique())
print("data2 sites:", data2["site"].unique())

data sites: ['HR1' 'HV1' 'ST4' 'LB4']
data2 sites: ['MY1' 'BG2' 'BL0' 'TH2' 'BX1']


In [17]:
site_cols = [c for c in combined.columns if c.lower() in ["lat","latitude","lon","longitude","x","y","easting","northing"]]
site_cols


[]

In [18]:
import os
print("Files in runtime:", os.listdir("."))


Files in runtime: ['.config', 'LaqnData (2).csv', 'LaqnData (3).csv', 'sample_data']


In [19]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.listdir('/content/drive/MyDrive')


Mounted at /content/drive


['uni applications',
 'Jayne Kranat - Cambridge International AS & A Level Mathematics_ Probability & Statistics 2-CUP (2018).pdf',
 'chem',
 '11 October 1987.gdoc',
 'maths pp.gdoc',
 'LDJITN Quote bank .docx',
 'hamlet quote bank .docx',
 'paragraphs .docx',
 'quote bank .docx',
 'eng lit a level',
 'internships ',
 'uni',
 'evidence in the effectiveness of wind turbines and solar panels.gdoc',
 'evidence in the effectiveness of wind turbines and solar panels.docx',
 'tutorial 5.docx',
 'Homework 3.docx',
 'homework 1.docx',
 'ATK superconcept essay  (1).docx',
 'ATK superconcept essay .docx',
 'Individual Report.docx',
 'IRM Final.gdoc',
 'individual reflective practice .docx',
 'industrial strategy.gdoc',
 'Untitled document.gdoc',
 '1. Asthma + Lung UK.gdoc',
 'contacts  .gdoc',
 'Copy of Ana Stefan CV .docx',
 'bank of america.gdoc',
 'Colab Notebooks',
 'Spanish essay.gdoc',
 'Spanish project.gdoc']

In [23]:
from google.colab import files
uploaded = files.upload()

Saving LondonWideUltraLowEmissionZone.geojson to LondonWideUltraLowEmissionZone.geojson


In [28]:
import geopandas as gpd, pandas as pd, numpy as np
import statsmodels.formula.api as smf


site_coords = pd.DataFrame({
    'site': ['HR1', 'HV1', 'ST4', 'LB4', 'MY1', 'BG2', 'BL0', 'TH2', 'BX1'],
    'latitude': [51.52, 51.48, 51.55, 51.50, 51.51, 51.49, 51.50, 51.53, 51.60],
    'longitude': [-0.10, -0.05, -0.15, 0.02, -0.07, 0.00, -0.10, -0.12, -0.20]
})


pts = gpd.GeoDataFrame(site_coords, geometry=gpd.points_from_xy(site_coords.longitude, site_coords.latitude), crs="EPSG:4326").to_crs(27700)


ulez = gpd.read_file("LondonWideUltraLowEmissionZone.geojson").to_crs(27700)

poly = ulez.geometry.buffer(0).union_all()


pts["dist_m"] = pts.geometry.apply(lambda p: -p.distance(poly.boundary) if poly.contains(p) else p.distance(poly.boundary))
pts["inside"] = (pts["dist_m"] < 0).astype(int)

df = combined.merge(pts[["site","dist_m","inside"]], on="site", how="left")

df["year"] = df["datetime"].dt.year

post = df[df["year"].eq(2020)].groupby(["site","dist_m","inside"], as_index=False)["Value"].mean()


h = 15000
rd = post[post["dist_m"].abs() <= h].assign(x=lambda d: d["dist_m"])
res = smf.ols("Value ~ inside + x + inside:x", data=rd).fit(cov_type="HC1")
print(res.summary())
print("RD jump (at boundary) =", res.params["inside"])


                            OLS Regression Results                            
Dep. Variable:                  Value   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.138
Method:                 Least Squares   F-statistic:                     24.46
Date:                Thu, 08 Jan 2026   Prob (F-statistic):           0.000694
Time:                        18:03:47   Log-Likelihood:                -36.441
No. Observations:                   9   AIC:                             76.88
Df Residuals:                       7   BIC:                             77.28
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     17.3756     14.565      1.193      0.2

/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 2
  warnings.warn('covariance of constraints does not have full '


In [29]:
res = smf.ols(
    "Value ~ inside + x",
    data=rd
).fit(cov_type="HC1")
